In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
url = 'https://raw.githubusercontent.com/fernandoawari/calculadora-aluguel-turma-set-20/main/1-web-scraping/dados_aluguel_ml_sao_paulo.csv'

In [3]:
df = pd.read_csv(url, thousands='.')

In [4]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel
0,norte,50 m² construídos2 quartos,"Praça Rafael Mendes De Carvalho, Jardim Paraís...",1150
1,norte,76 m² construídos3 quartos,"Rua Almirante Noronha, Jardim São Paulo, São P...",1480
2,norte,30 m² construídos1 quarto,"Rua Cássio De Almeida, Vila Guilherme, São Pau...",800
3,norte,80 m² construídos3 quartos,"Rua Cabo José Da Silva, Parque Novo Mundo, São...",1180
4,norte,70 m² construídos2 quartos,"Parque Domingos Luís, Jardim São Paulo, São Pa...",1300


In [5]:
df.shape

(1344, 4)

In [6]:
# Vamos listar para ver possíveis problemas (valores ausentes que ocorrem)
list(df['area_quartos'])

['50 m² construídos2 quartos',
 '76 m² construídos3 quartos',
 '30 m² construídos1 quarto',
 '80 m² construídos3 quartos',
 '70 m² construídos2 quartos',
 '45 m² construídos1 quarto',
 '65 m² construídos2 quartos',
 '55 m² construídos2 quartos',
 '49 m² construídos2 quartos',
 '70 m² construídos1 quarto',
 '74 m² construídos3 quartos',
 '50 m² construídos2 quartos',
 '60 m² construídos2 quartos',
 '54 m² construídos2 quartos',
 '92 m² construídos3 quartos',
 '74 m² construídos3 quartos',
 '80 m² construídos2 quartos',
 '57 m² construídos2 quartos',
 '89 m² construídos2 quartos',
 '52 m² construídos2 quartos',
 '110 m² construídos3 quartos',
 '56 m² construídos2 quartos',
 '53 m² construídos2 quartos',
 '53 m² construídos2 quartos',
 '60 m² construídos3 quartos',
 '59 m² construídos3 quartos',
 '78 m² construídos2 quartos',
 '80 m² construídos2 quartos',
 '330 m² construídos4 quartos',
 '40 m² construídos2 quartos',
 '50 m² construídos2 quartos',
 '300 m² construídos3 quartos',
 '194 m²

In [7]:
# Criei esse subset com algumas das situacoes que encontramos
area_quartos_subset = ['1,360 m² totais',
                       '2 quartos',
                       '53 m² construídos2 quartos',
                        '280 m² construídos',
                        '2,600 m² construídos',
                       ]

In [8]:
# Vamos agora testar criar uma funcao para extracao da área. Primeiramente testando passo a passo
entrada = '1,360 m² totais'

In [9]:
entrada.split(' m²')[0]

'1,360'

In [10]:
area = entrada.split(' m²')[0]

In [11]:
area.replace(',', '')

'1360'

In [12]:
if ' m²' in entrada:
  area = entrada.split(' m²')[0]
  area = area.replace(',', '')
else:
  area = np.nan

In [13]:
def extrair_area(entrada):
  """Criar função que possua como entrada um texto e retorne a área contida no
  mesmo. Retornar np.nan se for ausentes. Alguns exemplos:
      >>> extrair_area('79 m² construídos1 quarto')
      >>> 79
      >>> extrair_area('280 m² construídos')
      >>> 280
      >>> extrair_area('2,600 m² construídos'
      >>> 2600
      >>> extrair_area('2 quartos')
      >>> np.nan
      """
  if ' m²' in entrada:
    area = entrada.split(' m²')[0]
    area = area.replace(',', '')
  else:
    area = np.nan
  return area

In [14]:
for aq in area_quartos_subset:
  print(extrair_area(aq))

1360
nan
53
280
2600


In [15]:
# Aplicando a funcao para a coluna area_quartos
df['area']=df['area_quartos'].apply(extrair_area).astype(float)

In [16]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel,area
0,norte,50 m² construídos2 quartos,"Praça Rafael Mendes De Carvalho, Jardim Paraís...",1150,50.0
1,norte,76 m² construídos3 quartos,"Rua Almirante Noronha, Jardim São Paulo, São P...",1480,76.0
2,norte,30 m² construídos1 quarto,"Rua Cássio De Almeida, Vila Guilherme, São Pau...",800,30.0
3,norte,80 m² construídos3 quartos,"Rua Cabo José Da Silva, Parque Novo Mundo, São...",1180,80.0
4,norte,70 m² construídos2 quartos,"Parque Domingos Luís, Jardim São Paulo, São Pa...",1300,70.0


In [17]:
# Repetindo o mesmo, só que agora para extração do número de quartos
area_quartos_subset = ['1,360 m² totais',
                       '2 quartos',
                       '1 quarto',
                       '53 m² construídos2 quartos',
                        '280 m² construídos',
                        '2,600 m² construídos',
                       ]

In [18]:
entrada='2 quartos'

In [19]:
# Neste caso vamos precisar usar regex
re.findall('(.?) quarto', entrada)[0]

'2'

In [20]:
def extrair_quartos(entrada):
  """Criar função que possua como entrada um texto e retorne uma tupla com a 
  área e o quarto. Retornar np.nan se um dos dois for ausentes. Alguns exemplos:
      >>> extrair_quarto('79 m² construídos1 quarto')
      >>> 1
      >>> extrair_quarto('280 m² construídos')
      >>> np.nan
      >>> extrair_quarto('2,600 m² construídos'
      >>> np.nan
      >>> extrair_quarto('2 quartos')
      >>> 2"""
  if 'quarto' in entrada:
    n_quartos = re.findall('(\d*) quarto', entrada)[0]
  else:
    n_quartos = np.nan
  return n_quartos

In [21]:
area_quartos_subset = ['1,360 m² totais',
                       '2 quartos',
                       '10 quarto',
                       '53 m² construídos2 quartos',
                        '280 m² construídos',
                        '2,600 m² construídos',
                       ]

In [22]:
for aq in area_quartos_subset:
  print(extrair_quartos(aq))

nan
2
10
2
nan
nan


In [23]:
df['quartos'] = df['area_quartos'].apply(extrair_quartos).astype(float)

In [24]:
# removendo a coluna area_quartos
df = df.drop('area_quartos', axis=1)

In [25]:
# reposicionando as colunas
cols = ['zona', 'enderecos_lista', 'area', 'quartos', 'aluguel']

In [26]:
df=df[cols]

In [27]:
df.head()

,zona,enderecos_lista,area,quartos,aluguel
0,norte,"Praça Rafael Mendes De Carvalho, Jardim Paraís...",50.0,2.0,1150
1,norte,"Rua Almirante Noronha, Jardim São Paulo, São P...",76.0,3.0,1480
2,norte,"Rua Cássio De Almeida, Vila Guilherme, São Pau...",30.0,1.0,800
3,norte,"Rua Cabo José Da Silva, Parque Novo Mundo, São...",80.0,3.0,1180
4,norte,"Parque Domingos Luís, Jardim São Paulo, São Pa...",70.0,2.0,1300


In [28]:
# antes de movermos para preprocessamentos voltados à valores ausentes, vamos 
# salvar os dados do jeito como estão
df.to_csv('dados_aluguel_ml_sao_paulo_preprocessado.csv', index=False)

In [29]:
# Index=false usado anteriormente para que o csv seja carregado sem necessitar argumentos adicionais
pd.read_csv('dados_aluguel_ml_sao_paulo_preprocessado.csv')

,zona,enderecos_lista,area,quartos,aluguel
0,norte,"Praça Rafael Mendes De Carvalho, Jardim Paraís...",50.0,2.0,1150
1,norte,"Rua Almirante Noronha, Jardim São Paulo, São P...",76.0,3.0,1480
2,norte,"Rua Cássio De Almeida, Vila Guilherme, São Pau...",30.0,1.0,800
3,norte,"Rua Cabo José Da Silva, Parque Novo Mundo, São...",80.0,3.0,1180
4,norte,"Parque Domingos Luís, Jardim São Paulo, São Pa...",70.0,2.0,1300
...,...,...,...,...,...
1339,oeste,"Rua Piauí, Higienópolis, São Paulo Zona Oeste,...",480.0,4.0,13280
1340,oeste,"Rua Doutor Aires Martins Torres, Vila São Fran...",620.0,4.0,15000
1341,oeste,"Rua Doutor Martins De Oliveira, Jardim Londrin...",64.0,3.0,2390
1342,oeste,"Rua Natingui, Vila Madalena, São Paulo Zona Oe...",44.0,1.0,3600


**Lidando com Dados Ausentes**

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1344 entries, 0 to 1343
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   zona             1344 non-null   object 
 1   enderecos_lista  1344 non-null   object 
 2   area             1341 non-null   float64
 3   quartos          1273 non-null   float64
 4   aluguel          1344 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 52.6+ KB


In [31]:
# uma opção é remover todos os valores ausentes
df.dropna()

,zona,enderecos_lista,area,quartos,aluguel
0,norte,"Praça Rafael Mendes De Carvalho, Jardim Paraís...",50.0,2.0,1150
1,norte,"Rua Almirante Noronha, Jardim São Paulo, São P...",76.0,3.0,1480
2,norte,"Rua Cássio De Almeida, Vila Guilherme, São Pau...",30.0,1.0,800
3,norte,"Rua Cabo José Da Silva, Parque Novo Mundo, São...",80.0,3.0,1180
4,norte,"Parque Domingos Luís, Jardim São Paulo, São Pa...",70.0,2.0,1300
...,...,...,...,...,...
1339,oeste,"Rua Piauí, Higienópolis, São Paulo Zona Oeste,...",480.0,4.0,13280
1340,oeste,"Rua Doutor Aires Martins Torres, Vila São Fran...",620.0,4.0,15000
1341,oeste,"Rua Doutor Martins De Oliveira, Jardim Londrin...",64.0,3.0,2390
1342,oeste,"Rua Natingui, Vila Madalena, São Paulo Zona Oe...",44.0,1.0,3600


In [32]:
# Ou preencher com um valor constante
df.fillna(-1)

,zona,enderecos_lista,area,quartos,aluguel
0,norte,"Praça Rafael Mendes De Carvalho, Jardim Paraís...",50.0,2.0,1150
1,norte,"Rua Almirante Noronha, Jardim São Paulo, São P...",76.0,3.0,1480
2,norte,"Rua Cássio De Almeida, Vila Guilherme, São Pau...",30.0,1.0,800
3,norte,"Rua Cabo José Da Silva, Parque Novo Mundo, São...",80.0,3.0,1180
4,norte,"Parque Domingos Luís, Jardim São Paulo, São Pa...",70.0,2.0,1300
...,...,...,...,...,...
1339,oeste,"Rua Piauí, Higienópolis, São Paulo Zona Oeste,...",480.0,4.0,13280
1340,oeste,"Rua Doutor Aires Martins Torres, Vila São Fran...",620.0,4.0,15000
1341,oeste,"Rua Doutor Martins De Oliveira, Jardim Londrin...",64.0,3.0,2390
1342,oeste,"Rua Natingui, Vila Madalena, São Paulo Zona Oe...",44.0,1.0,3600


In [33]:
# Ou ainda preencher com média ou mediana
df.describe()

,area,quartos,aluguel
count,1341.000000,1273.000000,1344.000000
mean,112.697987,2.106834,4396.404762
std,259.503149,0.882869,29225.842756
min,1.000000,1.000000,299.000000
25%,48.000000,1.000000,1500.000000
50%,67.000000,2.000000,2200.000000
75%,106.000000,3.000000,3577.500000
max,7800.000000,5.000000,870000.000000


In [34]:
df.median()

area         67.0
quartos       2.0
aluguel    2200.0
dtype: float64

In [35]:
df.fillna(df.median())

,zona,enderecos_lista,area,quartos,aluguel
0,norte,"Praça Rafael Mendes De Carvalho, Jardim Paraís...",50.0,2.0,1150
1,norte,"Rua Almirante Noronha, Jardim São Paulo, São P...",76.0,3.0,1480
2,norte,"Rua Cássio De Almeida, Vila Guilherme, São Pau...",30.0,1.0,800
3,norte,"Rua Cabo José Da Silva, Parque Novo Mundo, São...",80.0,3.0,1180
4,norte,"Parque Domingos Luís, Jardim São Paulo, São Pa...",70.0,2.0,1300
...,...,...,...,...,...
1339,oeste,"Rua Piauí, Higienópolis, São Paulo Zona Oeste,...",480.0,4.0,13280
1340,oeste,"Rua Doutor Aires Martins Torres, Vila São Fran...",620.0,4.0,15000
1341,oeste,"Rua Doutor Martins De Oliveira, Jardim Londrin...",64.0,3.0,2390
1342,oeste,"Rua Natingui, Vila Madalena, São Paulo Zona Oe...",44.0,1.0,3600


In [36]:
df.to_csv('dados_aluguel_ml_sao_paulo_preprocessado.csv')

In [37]:
df

,zona,enderecos_lista,area,quartos,aluguel
0,norte,"Praça Rafael Mendes De Carvalho, Jardim Paraís...",50.0,2.0,1150
1,norte,"Rua Almirante Noronha, Jardim São Paulo, São P...",76.0,3.0,1480
2,norte,"Rua Cássio De Almeida, Vila Guilherme, São Pau...",30.0,1.0,800
3,norte,"Rua Cabo José Da Silva, Parque Novo Mundo, São...",80.0,3.0,1180
4,norte,"Parque Domingos Luís, Jardim São Paulo, São Pa...",70.0,2.0,1300
...,...,...,...,...,...
1339,oeste,"Rua Piauí, Higienópolis, São Paulo Zona Oeste,...",480.0,4.0,13280
1340,oeste,"Rua Doutor Aires Martins Torres, Vila São Fran...",620.0,4.0,15000
1341,oeste,"Rua Doutor Martins De Oliveira, Jardim Londrin...",64.0,3.0,2390
1342,oeste,"Rua Natingui, Vila Madalena, São Paulo Zona Oe...",44.0,1.0,3600
